In [1]:
! pkill -f chromedriver

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from pathlib import Path

In [16]:
# **1. Setup WebDriver**
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-cache") 
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
options.add_argument("--disable-blink-features=AutomationControlled")

In [17]:
# Inisialisasi WebDriver Chrome
driver = webdriver.Chrome(service=service, options=options)

# Baca file CSV yang berisi link artikel
csv_path = "/Users/mackumara/Dev/AI/Topic-Modelling-Article-Titles/data/raw/article_links.csv"
df = pd.read_csv(csv_path)

# Inisialisasi List untuk Menyimpan Hasil Scraping
all_titles = []  

all_years = []  

# Atur Maksimal Halaman yang Akan Di-Scrape
MAX_PAGES = 25  # Ubah sesuai kebutuhan

for index, row in df.iterrows():
    try:
        base_url = row['URL']  
        print(f"Scraping artikel dari: {base_url}")

        # Looping melalui setiap halaman
        for page_number in range(1, MAX_PAGES + 1):
            # Tambahkan parameter pagination ke URL
            url = f"{base_url}&sortType=vol-only-newest&pageNumber={page_number}"
            print(f"Membuka halaman {page_number}: {url}")

            # Buka Halaman IEEE Xplore
            driver.get(url)

            # Tunggu agar halaman selesai dimuat
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CLASS_NAME, "col"))
            ) 
            # Beri jeda waktu untuk memastikan halaman sepenuhnya dimuat
            time.sleep(5)

            # Parsing Halaman dengan BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Cari Semua Artikel dalam <div class="col result-item-align px-3">
            articles = soup.find_all('div', class_='col result-item-align px-3')

            # Jika tidak ada artikel, hentikan looping halaman
            if not articles:
                print(f"Tidak ada artikel di halaman {page_number}. Berhenti.")
                break

            # Looping untuk Mencari Judul & Tahun dalam Kontainer
            for article in articles:
                # Cari judul artikel dalam <h2> -> <a>
                h2_tag = article.find('h2')  # Cari elemen <h2>
                title_tag = h2_tag.find('a') if h2_tag else None  # Cari <a> di dalam <h2>
                title = title_tag.text.strip() if title_tag else "Judul Tidak Ditemukan"
                all_titles.append(title)  # Simpan judul dalam list

                # Cari tahun publikasi dalam <div class_="description text-base-md-lh"> -> <span>
                year_container = article.find('div', class_='description text-base-md-lh')  
                year_tag = year_container.find('span') if year_container else None
                year = year_tag.text.strip() if year_tag else "Tahun Tidak Ditemukan"
                all_years.append(year)

            print(f"Berhasil scraping {len(articles)} artikel dari halaman {page_number}")

    except KeyError as e:
        print(f"Error: Kolom 'link' tidak ditemukan. Pastikan nama kolom sesuai. Error: {e}")
    except Exception as e:
        print(f"Terjadi kesalahan saat scraping {url}: {e}")

driver.quit()

Scraping artikel dari: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=7419931&punumber=6287639
Membuka halaman 1: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=7419931&punumber=6287639&sortType=vol-only-newest&pageNumber=1
Berhasil scraping 25 artikel dari halaman 1
Membuka halaman 2: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=7419931&punumber=6287639&sortType=vol-only-newest&pageNumber=2
Berhasil scraping 25 artikel dari halaman 2
Membuka halaman 3: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=7419931&punumber=6287639&sortType=vol-only-newest&pageNumber=3
Berhasil scraping 25 artikel dari halaman 3
Membuka halaman 4: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=7419931&punumber=6287639&sortType=vol-only-newest&pageNumber=4
Berhasil scraping 25 artikel dari halaman 4
Membuka halaman 5: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=7419931&punumber=6287639&sortType=vol-only-newest&pageNumber=5
Berhasil scraping 25 artikel dari ha

In [19]:

# Buat DataFrame dari Hasil Scraping
output_df = pd.DataFrame({
    'Judul': all_titles,
    'Tahun': all_years,
})

# Tentukan Path untuk Menyimpan File CSV
output_csv_path = "/Users/mackumara/Dev/AI/Topic-Modelling-Article-Titles/data/raw/scraped_articles.csv"

# Buat direktori jika belum ada
output_dir = Path(output_csv_path).parent
output_dir.mkdir(parents=True, exist_ok=True) 

# Simpan DataFrame ke CSV
output_df.to_csv(output_csv_path, index=False)

print(f"\nHasil scraping telah disimpan ke: {output_csv_path}") 


Hasil scraping telah disimpan ke: /Users/mackumara/Dev/AI/Topic-Modelling-Article-Titles/data/raw/scraped_articles.csv


In [21]:
# Cleaning data

import pandas as pd

# Baca file CSV
csv_path = "/Users/mackumara/Dev/AI/Topic-Modelling-Article-Titles/data/raw/scraped_articles.csv"
df = pd.read_csv(csv_path)

# Hapus teks 'Publication Year: ' dari kolom Tahun
df['Tahun'] = df['Tahun'].str.replace('Publication Year: ', '', regex=False)

# Simpan kembali file CSV yang sudah diperbarui
output_csv_path = "/Users/mackumara/Dev/AI/Topic-Modelling-Article-Titles/data/clean_title_articles.csv"
df.to_csv(output_csv_path, index=False)

print(f"File CSV yang sudah dibersihkan telah disimpan ke: {output_csv_path}")


File CSV yang sudah dibersihkan telah disimpan ke: /Users/mackumara/Dev/AI/Topic-Modelling-Article-Titles/data/clean_title_articles.csv
